# MVD 4. cvičení

## 1. část - Načtení dat

Po rozbalení archive.zip uvidíte articles csv soubor. Tento soubor pochází z [Kaggle datasetů](https://www.kaggle.com/hsankesara/medium-articles) a obsahuje malé množství Medium článků k tématům ML, AI a data science. K úloze dnešního cvičení bude stačit využítí dat s názvy a obsahy článků (title a text).


### Příprava dat

Pro přípravu dat se použivá různá sekvence kroků. Je doporučeno na následující kroky vytvořit samostatnou funkci, aby bylo možné zpracovat i vyhledávaný výraz při testování. Dnešní cvičení by mělo obsahovat následující kroky:

1. Převést všechen text na lower case
2. Odstranění interpunkce a všech speciálních znaků (apostrof, ...)
3. Aplikace lemmatizátoru

Pozn.: Jedná se pouze o jednoduchý preprocessing, v praxi je často potřeba použití více kroků. Tato aplikace by měla například problém s čísly (desetinná čísla, čísla vyhledávaná slovně). 

Pro lemmatizaci použijte knihovnu spaCy.

In [1]:
# Instalace spaCy z Jupyter Notebooku
import sys
!{sys.executable} -m pip install spacy

# Stažení modelu pro angličtinu
!{sys.executable} -m spacy download en

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 5.9 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.5/806.5 kB 13.3 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 3.9 MB/s eta 0:00:004.7 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 kB 5.2 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 3.7 MB/s eta 0:00:004.2 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 9.9 MB/s eta 0:00:000m eta 0:00:010:0101
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 12.5 MB/s eta 0:00:00m eta 0:00:0136m0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 3.9 MB/s et

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [59]:
import spacy
import csv
import re
import math

lemmatizer = spacy.load('en_core_web_sm', disable=['parser', 'ner']) # NLTK
# Lemmatizace textu př.:  
# " ".join([token.lemma_ for token in lemmatizer(text)])

/home/filip/miniconda3/envs/mvd/lib/python3.10/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [83]:
def load_data(file_path: str) -> tuple:
    with open(file_path, 'rt') as file:
        reader = csv.reader(file, delimiter=',')
        data = [article for article in reader]
        return (data[0], data[1:])
    
def convert_to_lower(fields_id: list, articles: list) -> None:
    for article in articles:
        for field_id in fields_id:
            article[field_id] = article[field_id].lower()
            
def remove_chars(fields_id: list, articles: list, to_be_removed: re.Pattern) -> None:
    for article in articles:
        for field_id in fields_id:
            article[field_id] = re.sub(to_be_removed, '', article[field_id])
            article[field_id] = re.sub('\s', ' ', article[field_id])

def lemmatize(fields_id: list, articles: list, lemmatizer: spacy.lang.en.English) -> None:
    for article in articles:
        for field_id in fields_id:
            article[field_id] = " ".join([token.lemma_ for token in lemmatizer(article[field_id])])
            
header, data = load_data('archive/articles.csv')

convert_to_lower([4, 5], data)
to_be_removed = r'[.,?!/\\\"`\-:()\[\]*|—’–]'
remove_chars([4, 5], data, to_be_removed)
lemmatize([4, 5], data, lemmatizer)

## 2. část - Vytvoření invertovaného indexu

Před další prací s textem je potřeba vytvořit invertovaný index, který poté usnadní práci. Invertovaný index bude slovník, kde klíčem bude slovo a hodnotou bude list s id dokumentů (index), které dané slovo obsahují.

Pozn.: Je potřeba vytvořit dva invertované indexy - jeden pro title a druhý pro text.

In [89]:
def create_index(field_id: int, articles: list) -> dict:
    index = {}
    
    for i, article in enumerate(articles):
        words = [word.strip() for word in re.split(r'\s', article[field_id]) if word != '']
        
        for word in words:
            if word in index:
                index[word].append(i)
            else:
                index[word] = [i]
    
    return index

title_index = create_index(4, data)
content_index = create_index(5, data)

## 3. část - Implementace TF-IDF

Připravení funkce pro výpočet TF-IDF po příchodu dotazu. Funkce *tf_idf* by měla pracovat s dotazem, jedním invertovaným indexem a s danými dokumenty. Vrátit by měla list obsahující skóre pro každý dokument.

<br>
<center>
$
score(q,d) = TF\_IDF(q,d) = \sum\limits_{w \in q \cap d} c(w, q) c(w, d) log(\frac{M+1}{df(w)})
$
</center>

$q$ ... dotaz<br>
$d$ ... dokument<br>
$c(w, q)$ ... kolikrát je slovo *w* v dotazu *q*<br>
$M$ ... celkový počet dokumentů<br>
$df(w)$ ... počet dokumentů, ve kterých se nachází slovo *w*

In [85]:
def parse_query(query_string: str) -> tuple:
    query = [[query_string]]
    
    convert_to_lower([0], query)
    to_be_removed = r'[.,?!/\\\"`\-:()\[\]*|—’–]'
    remove_chars([0], query, to_be_removed)
    lemmatize([0], query, lemmatizer)
    
    return query[0][0], [word.strip() for word in re.split(r'\s', query[0][0]) if word != '']

def compute_tf_idf(query_string: str, data: list, field_id: int, index: dict) -> list:
    query_lemmatized, query_lemmatized_list = parse_query(query_string)
    scores = []
    
    for document in data:
        document_words = [word.strip() for word in re.split(r'\s', document[field_id]) if word != '']
        common_words = set(query_lemmatized_list).intersection(set(document_words))
        score = 0
        
        for word in common_words:
            word_count_in_query = query_lemmatized.count(word)
            word_count_in_doc = document[field_id].count(word)
            
            score += word_count_in_query * word_count_in_doc * math.log(len(data) + 1) / len(title_index[word])

        scores.append(score)
        
    return scores

## 4. část - Použití a testování TF-IDF

Nyní lze získat skóre pro titulky nebo text. Následujícím krokem je sjednocení výsledného skóre pro ohodnocení celého dokumentu. V případě dvou hodnot si vystačíme s parametrem $\alpha$, který nám určuje jakou váhu má titulek a jakou samotný text dokumentu. <br>

<center>
$
score(q,d) = \alpha \; TF\_IDF\_title(q,d) + (1-\alpha) \; TF\_IDF\_text(q,d)
$
</center>

Při nastavení parametru $\alpha$ na hodnotu 0.7 a vyhledávání dotazu "coursera vs udacity machine learning" by výsledky měly vypadat následovně:

![output](sample_output.png)

In [91]:
import numpy as np

alpha = 0.7
q = 'coursera vs udacity machine learning'

title_scores = np.array(compute_tf_idf(q, data, 4, title_index))
content_scores = np.array(compute_tf_idf(q, data, 5, content_index))

scores = alpha * title_scores + (1 - alpha) * content_scores
order = np.argsort(-scores)

print('{:3}  {:40}     {:40}     {:3.5}'.format('', 'title', 'text', 'scores'))

for i in order[:15]:
    print('{:3}  {:40}...  {:40}...  {:3.5}'.format(i, data[i][4][0:40], data[i][5][0:40], scores[i]))


     title                                        text                                         score
276  coursera vs udacity for machine learning...  2018 be an exciting time for student of ...  63.716
 67  every single machine learning course on ...  a year and a half ago I drop out of one ...  42.083
 99  every single machine learning course on ...  a year and a half ago I drop out of one ...  42.083
143  every single machine learning course on ...  a year and a half ago I drop out of one ...  42.083
 19  every single machine learning course on ...  a year and a half ago I drop out of one ...  42.083
 47  machine learning in a week   learn new s...  get into machine learning ml can seem li...  7.5647
  9  what I learn from interview at multiple ...  over the past 8 month I ve be interview ...  7.2918
160  what I learn from interview at multiple ...  over the past 8 month I ve be interview ...  7.2918
 93  what I learn from interview at multiple ...  over the past 8 month I ve be int